In [23]:
import numpy as np
import pandas as pd
import csv
import re
import time

In [24]:
rawdata = pd.read_csv('raw_data/training_set_no_time.csv', encoding = "ISO-8859-1")

In [25]:
rawdata.head()

,Engagements,Followers at Posting,Type,Description
0,502093,36984682,Video,The @raptors bench trio of @sergeibaka @norman...
1,603380,36984682,Video,@kyle_lowry7 pulls from deep for the @raptors ...
2,603380,36984682,Video,@k_mid22 with some english on the @bucks dime!
3,725100,36984682,Video,Kawhi punches it home with the left on TNT!
4,661446,36984682,Video,@giannis_an34 goes baseline early to rock the ...


In [26]:
rawdata['len_text'] = list(map(lambda x: len(str(x)), rawdata['Description']))

In [27]:
rawdata['word_ct'] = list(map(lambda x: str(x).count(' '), rawdata['Description']))

In [28]:
rawdata['excl_ct'] = list(map(lambda x: str(x).count('!'), rawdata['Description']))

In [29]:
rawdata['emoji_ct'] = list(map(lambda x: str(x).count('??'), rawdata['Description']))

In [50]:
rawdata['ques_ct'] = list(map(lambda x: str(x).replace("??", "").count('?'), rawdata['Description']))

In [30]:
rawdata['period_ct'] = list(map(lambda x: str(x).count('.'), rawdata['Description']))

In [31]:
rawdata['comma_ct'] = list(map(lambda x: str(x).count(','), rawdata['Description']))

In [32]:
rawdata['caps_ct'] = list(map(lambda x: len(re.findall(r'[A-Z]', str(x))), rawdata['Description']))

In [33]:
rawdata['tags_ct'] = list(map(lambda x: str(x).count('@'), rawdata['Description']))

In [34]:
import re
def hasTag(y):
    if (y is not None):
        if len(y) > 0:
            if (y[0] == '@'):
                return True
    else:
        return False

In [56]:
raw_players = list(map(lambda x: list(filter(lambda y: hasTag(y), str(x).split())), rawdata['Description']))

In [57]:
raw_players

[['@raptors', '@sergeibaka', '@normanpowell4', '@fredvanvleet'],
 ['@kyle_lowry7', '@raptors'],
 ['@k_mid22', '@bucks'],
 [],
 ['@giannis_an34'],
 ['@lukadoncic', '@traeyoung!'],
 ['@warriors'],
 ['@bucks', '@raptors'],
 ['@warriors', '@nbastore!'],
 ['@stephencurry30...', '@nbahistory!'],
 ['@stephencurry30', '@money23green'],
 ['@warriors', '@trailblazers'],
 ['@money23green', '@warriors!'],
 ['@warriors'],
 ['@money23green', '@warriors'],
 ['@stephencurry30', '@money23green', '@nbahistory'],
 ['@meyersleonard11!'],
 ['@money23green', '@warriors'],
 ['@stephencurry30'],
 ['@stephencurry30', '@warriors'],
 ['@damianlillard', '@trailblazers'],
 ['@meyersleonard11', '@trailblazers'],
 ['@stephencurry30'],
 ['@money23green'],
 ['@budweiserusa', '@trailblazers', '@trailblazers', '@warriors?'],
 ['@money23green', '@warriors', '@trailblazers', '@nbaonespn'],
 ['@trailblazers...', '@warriors', '@nbaonespn!'],
 ['@raptors'],
 ['@raptors', '@kristenledlow!'],
 ['@raptors'],
 ['@raptors'],
 ['@

In [36]:
##regex = ' \s+\n,.-_!?/\\><()&'
regex = "[\s\n\x91\x92,-/\\><()&!?\*^#|:;']+"
rawdata['player'] = list(map(lambda x: list(filter(lambda y: hasTag(y), re.split(regex, str(x)))), rawdata['Description']))

In [54]:
rawdata['player'].head()

0    [@raptors, @sergeibaka, @normanpowell4, @fredv...
1                             [@kyle_lowry7, @raptors]
2                                   [@k_mid22, @bucks]
3                                                   []
4                                      [@giannis_an34]
Name: player, dtype: object

In [38]:
teams = pd.read_csv('raw_data/teams.csv')
teams.head()

,Team Name,Insta_ID,Number of Posts,Number of Followers
0,Atlanta Hawks,atlhawks,7336,1M
1,Boston Celtics,celtics,7822,4.2M
2,Brooklyn Nets,brooklynnets,6363,1.2M
3,Charlotte Hornets,hornets,9653,1M
4,Chicago Bulls,chicagobulls,5202,4.4M


In [39]:
# function for pulling instagram followers:
import urllib.request, json 
def get_followers(searchquery):
    url = 'https://www.instagram.com/' + searchquery + '/?__a=1'
    with urllib.request.urlopen(url) as u:
        data = json.loads(u.read().decode())
        return data['graphql']['user']['edge_followed_by']['count']

In [43]:
# distinguish player from team:
def get_player_followers(searchquery):
    print(searchquery)
    time.sleep(2)
    if searchquery not in teams['Insta_ID']:
        followers = 0
        try:
            followers = get_followers(searchquery)
        except Exception:
            print("HAS ERROR")
        return followers
    return 0
# team:
def get_team_followers(searchquery):
    print(searchquery)
    time.sleep(2)
    if searchquery in teams['Insta_ID']:
        followers = 0
        try:
            followers = get_followers(searchquery)
        except Exception:
            print("HAS ERROR")
        return followers
    return 0

In [47]:
## Gather the follower count for all players:
player_list = []
rawdata['player'].values
for i in rawdata['player']:
    player_list = player_list + i
player_set = list(set(player_list))
print(len(player_set))
print(len(player_list))
## Gather count for all teams:
team_set = teams['Insta_ID']
print(len(team_set))

849
11713
30


In [48]:
# player fc
pfc = pd.DataFrame(player_set)
pfc.columns = ['player']
# team fc
tfc = pd.DataFrame(team_set)
tfc.columns = ['team']

In [21]:
pfc['fc'] = pfc['player'].apply(lambda x: get_player_followers(x.strip('@')))


sergeibaka
HAS ERROR
davidnwaba
HAS ERROR
blakeney96
HAS ERROR
Raptors905
HAS ERROR
shabazznap13r
HAS ERROR
Timberwolves
HAS ERROR
44bojan
HAS ERROR
omricasspi18
HAS ERROR
illwil21
HAS ERROR
lebrechtmedia
HAS ERROR
krisdunn3
HAS ERROR
troybrown
HAS ERROR
treyburke
HAS ERROR
justintimberlake
HAS ERROR
averybradleycamps
HAS ERROR
nbakicks
HAS ERROR
35_fitz
HAS ERROR
JHarden13
HAS ERROR
shanelarkin_0
HAS ERROR
minnesotalynx
HAS ERROR
dequanmjones
HAS ERROR
keefmorris5
HAS ERROR
greenranger14
HAS ERROR
kyrieirving
HAS ERROR
bradbeal3
HAS ERROR
LongIslandNets
HAS ERROR
kevinlove
HAS ERROR
nvonleh
HAS ERROR
nbacoaches
HAS ERROR
therea1djones
HAS ERROR
trey_duval
HAS ERROR
washmystics
HAS ERROR
fearthefro95
HAS ERROR
natlyphoto
HAS ERROR
kingrawle
HAS ERROR
_bigjayy_
HAS ERROR
channingfrye
HAS ERROR
eneskanter11
HAS ERROR
frenchmontana
HAS ERROR
kyle_lowry7
HAS ERROR
nicholascollison
HAS ERROR
juanchiviris41
HAS ERROR
bossmann99
HAS ERROR
oanunoby
HAS ERROR
22wiggins
HAS ERROR
andredrummondd


HAS ERROR
everybody_hatechriss
HAS ERROR
brian_babs_babineau
HAS ERROR
demarjaysmith
HAS ERROR
lsubasketball
HAS ERROR
terrellowens
HAS ERROR
skal_lab
HAS ERROR
_kw15
HAS ERROR
Raptors
HAS ERROR
hunter_games31
HAS ERROR
mollysullivantv
HAS ERROR
SacramentoKings
HAS ERROR
MiamiHEAT
HAS ERROR
damianlillard
HAS ERROR
lesdogggg
HAS ERROR
rasualbutler8
HAS ERROR
mbeli21
HAS ERROR
PelicansNBA
HAS ERROR
all
HAS ERROR
des_linden
HAS ERROR
spencerdinwiddie
HAS ERROR
bucks
HAS ERROR
jeremylamb1
HAS ERROR
matthewdelly
HAS ERROR
7tlc
HAS ERROR
markellefultz
HAS ERROR
adr13nsanmiguel
HAS ERROR
FIBA
HAS ERROR
j_lay_lay
HAS ERROR
justholla7
HAS ERROR
andre
HAS ERROR
orlandomagic
HAS ERROR
dtheis10
HAS ERROR
de11edonne
HAS ERROR
macbo50
HAS ERROR
UtahJazz
HAS ERROR
furkankorkkmaz
HAS ERROR
NBAHistory
HAS ERROR
bronzewhale
HAS ERROR
Paulmillsap4
HAS ERROR
mentornmp
HAS ERROR
stancehoops
HAS ERROR
stevenadams
HAS ERROR
mbeasy5
HAS ERROR
deandreayton
HAS ERROR
cp3basketballacademy
HAS ERROR
arsenal
HAS E

HAS ERROR
biggametae
HAS ERROR
nbcsauthentic
HAS ERROR
NBAGLeague
HAS ERROR
hoodie5
HAS ERROR
utahjazz
HAS ERROR
d_dedmon3
HAS ERROR
luoldeng9
HAS ERROR
djjasonleroux
HAS ERROR
gustogummi88
HAS ERROR
therealiceman44
HAS ERROR
jarriusrobertson
HAS ERROR
briskuno
HAS ERROR
Zo
HAS ERROR
tarikbblack
HAS ERROR
KobeBryant
HAS ERROR
kingjames
HAS ERROR
KingJames
HAS ERROR
jaytatum0
HAS ERROR
spidadmitchell
HAS ERROR
wayne_elli
HAS ERROR
brooklynnets
HAS ERROR
zachlavine8
HAS ERROR
jeffteague
HAS ERROR
melvinjrr
HAS ERROR
gordongram
HAS ERROR
quese
HAS ERROR
LindsayWhalen
HAS ERROR
rudygay
HAS ERROR
charliescott
HAS ERROR
patconnaughton
HAS ERROR
nbacares
HAS ERROR
allie14quigs
HAS ERROR
buddylove242
HAS ERROR
pharrell
HAS ERROR
johnhenson31
HAS ERROR
nenehilario_42
HAS ERROR
cgray209
HAS ERROR
fskpart3
HAS ERROR
dtaegraham
HAS ERROR
kuzmakyle
HAS ERROR
reggie_jackson
HAS ERROR
raptors
HAS ERROR
jdot43
HAS ERROR
okarowhite
HAS ERROR
dwhite921
HAS ERROR
50mejri
HAS ERROR
gardenofdreamsfoundatio

In [28]:
tfc['fc'] = tfc['team'].apply(lambda x: get_team_followers(x.strip('@')))


j_rob12
HAS ERROR
luoldeng9


KeyboardInterrupt: 

In [ ]:
pfc.to_csv('player_follower_count.csv')

## Merge follower count w/ main data

## Train RF and assess w/ CV

## Train and assess keras NN (https://machinelearningmastery.com/regression-tutorial-keras-deep-learning-library-python/)

In [49]:
import matplotlib.pyplot as plt
plt.scatter(rawdata['word_ct'], rawdata['len_text'])
plt.show()

<Figure size 640x480 with 1 Axes>

In [24]:
teams.head()

,Team Name,Insta_ID,Number of Posts,Number of Followers
0,Atlanta Hawks,atlhawks,7336,1M
1,Boston Celtics,celtics,7822,4.2M
2,Brooklyn Nets,brooklynnets,6363,1.2M
3,Charlotte Hornets,hornets,9653,1M
4,Chicago Bulls,chicagobulls,5202,4.4M


In [21]:
from lxml import html
import requests
from lxml.html import fromstring, tostring


In [42]:
searchquery = 'stephencurry30'
url = 'https://www.instagram.com/' + searchquery + '/?__a=1'
page = requests.get(url)
tree = html.fromstring(page.content)


In [50]:
import urllib.request, json 
with urllib.request.urlopen(url) as u:
    data = json.loads(u.read().decode())
    print(data)

{'logging_page_id': 'profilePage_324599988', 'show_suggested_profiles': True, 'graphql': {'user': {'biography': '@vsonwatch', 'blocked_by_viewer': False, 'country_block': False, 'external_url': 'http://fb.me/StephenvsTheGame', 'external_url_linkshimmed': 'https://l.instagram.com/?u=http%3A%2F%2Ffb.me%2FStephenvsTheGame&e=ATO5G_MDgJUOJZ43Bq_Bb4jIO0lAw12fKgakgMOZlFULA1TTeH3FHag-vrLck7bIEgdMLV-I', 'edge_followed_by': {'count': 25661091}, 'followed_by_viewer': False, 'edge_follow': {'count': 648}, 'follows_viewer': False, 'full_name': 'Wardell Curry', 'has_channel': False, 'has_blocked_viewer': False, 'highlight_reel_count': 4, 'has_requested_viewer': False, 'id': '324599988', 'is_business_account': False, 'is_joined_recently': False, 'business_category_name': None, 'is_private': False, 'is_verified': True, 'edge_mutual_followed_by': {'count': 0, 'edges': []}, 'profile_pic_url': 'https://scontent-sea1-1.cdninstagram.com/vp/eac4ca641e352ef94e652fb39583f44a/5D859F9A/t51.2885-19/s150x150/2227

In [52]:
data['logging_page_id']

'profilePage_324599988'

In [53]:
data['graphql']

{'user': {'biography': '@vsonwatch',
  'blocked_by_viewer': False,
  'country_block': False,
  'external_url': 'http://fb.me/StephenvsTheGame',
  'external_url_linkshimmed': 'https://l.instagram.com/?u=http%3A%2F%2Ffb.me%2FStephenvsTheGame&e=ATO5G_MDgJUOJZ43Bq_Bb4jIO0lAw12fKgakgMOZlFULA1TTeH3FHag-vrLck7bIEgdMLV-I',
  'edge_followed_by': {'count': 25661091},
  'followed_by_viewer': False,
  'edge_follow': {'count': 648},
  'follows_viewer': False,
  'full_name': 'Wardell Curry',
  'has_channel': False,
  'has_blocked_viewer': False,
  'highlight_reel_count': 4,
  'has_requested_viewer': False,
  'id': '324599988',
  'is_business_account': False,
  'is_joined_recently': False,
  'business_category_name': None,
  'is_private': False,
  'is_verified': True,
  'edge_mutual_followed_by': {'count': 0, 'edges': []},
  'profile_pic_url': 'https://scontent-sea1-1.cdninstagram.com/vp/eac4ca641e352ef94e652fb39583f44a/5D859F9A/t51.2885-19/s150x150/22277378_1720913538216240_2580026733478543360_n.jpg

In [54]:
data['graphql']['user']['edge_followed_by']['count']

25661091

In [52]:
rawdata.head()

,Engagements,Followers at Posting,Type,Description,len_text,word_ct,excl_ct,ques_ct,period_ct,comma_ct,caps_ct,tags_ct,player
0,502093,36984682,Video,The @raptors bench trio of @sergeibaka @norman...,95,14,1,0,0,0,2,4,"[@raptors, @sergeibaka, @normanpowell4, @fredv..."
1,603380,36984682,Video,@kyle_lowry7 pulls from deep for the @raptors ...,64,11,1,0,0,0,3,2,"[@kyle_lowry7, @raptors]"
2,603380,36984682,Video,@k_mid22 with some english on the @bucks dime!,46,7,1,0,0,0,0,2,"[@k_mid22, @bucks]"
3,725100,36984682,Video,Kawhi punches it home with the left on TNT!,43,8,1,0,0,0,4,0,[]
4,661446,36984682,Video,@giannis_an34 goes baseline early to rock the ...,57,9,1,0,0,0,3,1,[@giannis_an34]
